##       **1. COMBINACIÓN DE MÚLTIPLES ARCHIVOS RASTER**

El presente cuaderno busca combinar diferentes datos raster en un archivo multibanda; se busca que el resultado tenga tanto el mismo tipo de dato como igual manejo de valores *"no data"*

## 1.1. Importar librerias y definir carpeta que contiene los raster

In [2]:
import os
import rasterio
import numpy as np

# Carpeta y nombre del archivo de salida
carpeta = 'C:/Users/Equipo/SemDocker/ProyectoProgramacionSIG/Data/Imagenes_25_07_12_v2'
salida_path = os.path.join(carpeta, 'stack_completo.tif')

# Archivos en orden deseado
archivos = [
    'DEM9377.tif',                                      # 1 banda
    'gedi_resampleado9377.tif',
    'mapbiomas_amacayacu_2023_resampleado9377.tif',     # 1 banda
    'SAR_PW_resampleado9377.tif',                       # 2 bandas
    'sentinel2_index_proye_resampleado9377.tif'         # N bandas
]

## 1.2 Configuración general y procesamiento de los archivos

**'float32'** es un tipo de dato de punto flotante de 32 bits, usado para datos continuos como elevación o índices. Esto asegura la consistencia de los datos en el unión final

El **'with'** asegura que el archivo se cierre correctamente después de su uso

**'profile_base'** servirá como plantilla para el archivo de salida, garantizando que el stack final tenga la misma georreferenciación y dimensiones que el DEM

**'profile_base.update'** actualiza el perfil base con el tipo de dato y el valor nodata que se desean para el archivo de salida.

**'if nodata_original is not None'** Comprueba si el archivo original tiene un valor nodata definido en sus metadatos.

Es **importante el orden** porque aquí se define que el primer archivo corresponde a la plantilla


In [6]:
# Configuración general
dtype_salida = 'float32'       # Tipo de dato común
nodata_comun = -9999           # Valor nodata común para todas las bandas
bandas_stack = []              # Lista de arrays a unir
nombres_bandas = []            # Lista para log de seguimiento

# Leer el perfil del DEM como plantilla
with rasterio.open(os.path.join(carpeta, archivos[0])) as dem_src:
    profile_base = dem_src.profile.copy()
    profile_base.update(dtype=dtype_salida, nodata=nodata_comun)

# Procesar todos los archivos en orden
for nombre in archivos:
    path = os.path.join(carpeta, nombre)
    try:
        with rasterio.open(path) as src:
            data = src.read().astype(dtype_salida)  # Convertir al tipo común
            nodata_original = src.nodata

            # Imputación de nodata
            if nodata_original is not None:
                data[data == nodata_original] = nodata_comun
            else:
                data[np.isnan(data)] = nodata_comun

            # Agregar al stack y registrar info
            bandas_stack.append(data)
            for i in range(data.shape[0]):
                nombres_bandas.append(f"{nombre} - Banda {i+1}")

            print(f' {nombre}: {data.shape[0]} banda(s) agregada(s).')

    except Exception as e:
        print(f' Error leyendo {nombre}: {e}')

 DEM9377.tif: 1 banda(s) agregada(s).
 gedi_resampleado9377.tif: 1 banda(s) agregada(s).
 mapbiomas_amacayacu_2023_resampleado9377.tif: 1 banda(s) agregada(s).
 SAR_PW_resampleado9377.tif: 2 banda(s) agregada(s).
 sentinel2_index_proye_resampleado9377.tif: 7 banda(s) agregada(s).


## 1.3 Concatenar bandas y actualizar metadatos

**np.vstack** apila verticalmente los datos que se han almacenado previamente en *'bandas_stack'*


In [10]:
# Concatenar bandas
stack_final = np.vstack(bandas_stack)  # Shape: (total_bandas, height, width)

# Actualizar metadatos
profile_base.update(count=stack_final.shape[0])

# Guardar imagen unificada
with rasterio.open(salida_path, 'w', **profile_base) as dst:
    dst.write(stack_final)

print(f'\n Imagen multibanda guardada como: {salida_path}')
print(f' Total de bandas combinadas: {stack_final.shape[0]}')


 Imagen multibanda guardada como: C:/Users/Equipo/SemDocker/ProyectoProgramacionSIG/Data/Imagenes_25_07_12_v2\stack_completo.tif
 Total de bandas combinadas: 12


## 1.4 Resumen de bandas

In [13]:
# (Opcional) Mostrar resumen de bandas
print("\n Detalle de bandas apiladas:")
for idx, nombre_banda in enumerate(nombres_bandas, start=1):
    print(f"  Banda {idx:02d}: {nombre_banda}")


 Detalle de bandas apiladas:
  Banda 01: DEM9377.tif - Banda 1
  Banda 02: gedi_resampleado9377.tif - Banda 1
  Banda 03: mapbiomas_amacayacu_2023_resampleado9377.tif - Banda 1
  Banda 04: SAR_PW_resampleado9377.tif - Banda 1
  Banda 05: SAR_PW_resampleado9377.tif - Banda 2
  Banda 06: sentinel2_index_proye_resampleado9377.tif - Banda 1
  Banda 07: sentinel2_index_proye_resampleado9377.tif - Banda 2
  Banda 08: sentinel2_index_proye_resampleado9377.tif - Banda 3
  Banda 09: sentinel2_index_proye_resampleado9377.tif - Banda 4
  Banda 10: sentinel2_index_proye_resampleado9377.tif - Banda 5
  Banda 11: sentinel2_index_proye_resampleado9377.tif - Banda 6
  Banda 12: sentinel2_index_proye_resampleado9377.tif - Banda 7
